<a href="https://colab.research.google.com/github/ilaydaozer/PasswordStrengthChecker/blob/main/PasswordStrengthChecker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

RandomForestClassifier algoritmasını kullanarak 'password' ve 'strength' sütunlarını içeren bir veri setinde kullanıcının girdiği şifreyi 'weak', 'medium' veya 'strong' olarak etiketleyen bir projedir.

Veri seti : https://www.kaggle.com/datasets/bhavikbb/password-strength-classifier-dataset

In [ ]:
import pandas as pd
import numpy as np

error_bad_lines bir DataFrame oluşturulurken kullanılan bir parametredir ve varsayılan olarak True olarak ayarlanır. Eğer bu parametre True olarak ayarlanırsa, pd.read_csv() işlevi, dosyanın okunmasında hatalarla karşılaştığında bir hata üretir ve DataFrame'i oluşturmayı durdurur. Ancak, error_bad_lines=False olarak ayarlanırsa, bu hatalı satırları atlar ve işleme devam eder. Bu, dosyada hatalı satırlar varsa, bunları görmezden gelmek ve geri kalan verileri okumaya devam etmek için kullanışlı bir parametredir. Bu, özellikle büyük veri dosyalarını işlerken yararlı olabilir, çünkü herhangi bir hatalı satırın tüm dosyanın okunmasını engellemesini önler.

In [ ]:
data = pd.read_csv("data.csv", error_bad_lines=False)

<ipython-input-3-020df2085ff2>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("data.csv", error_bad_lines=False)
Skipping line 2810: expected 2 fields, saw 5
Skipping line 4641: expected 2 fields, saw 5
Skipping line 7171: expected 2 fields, saw 5
Skipping line 11220: expected 2 fields, saw 5
Skipping line 13809: expected 2 fields, saw 5
Skipping line 14132: expected 2 fields, saw 5
Skipping line 14293: expected 2 fields, saw 5
Skipping line 14865: expected 2 fields, saw 5
Skipping line 17419: expected 2 fields, saw 5
Skipping line 22801: expected 2 fields, saw 5
Skipping line 25001: expected 2 fields, saw 5
Skipping line 26603: expected 2 fields, saw 5
Skipping line 26742: expected 2 fields, saw 5
Skipping line 29702: expected 2 fields, saw 5
Skipping line 32767: expected 2 fields, saw 5
Skipping line 32878: expected 2 fields, saw 5
Skipping line 35643: expected 2 fields

Bu veri setinde 'strength' sütununda;
0 şifrenin güçsüz, 1 orta, 2 ise güçlü olduğu anlamına geliyor.

In [ ]:
data.head()

,password,strength
0,kzde5577,1
1,kino3434,1
2,visi7k1yr,1
3,megzy123,1
4,lamborghin1,1


In [ ]:
data = data.dropna()

İleride test edeceğimiz şifrelerin gücünü ölçmek ve etiketlemek için 0,1 ve 2 değerlerini karşılık gelen kategorik veri tipine çevirdik.

In [ ]:
data["strength"] = data["strength"].map({0: "Weak",1: "Medium",2: "Strong"})

*data*.sample() ifadesi, DataFrame'den rastgele örneklemler almayı sağlıyor.

In [ ]:
data.sample(5)

,password,strength
566271,vinoth123,Medium
310710,goose95,Weak
636774,itivo259,Medium
347528,baffi7,Weak
452361,swati20,Weak


Modeli eğitmeye geçmeden önce şifreleri tokenlaştırmamız gerekiyor. Modelin şifrenin gücünü tahmin etmek için rakam, harf ve sembollerin kombinasyonlarından öğrenmesine ihtiyacımız var.
Aşağıdaki word fonksiyonu parametre olarak aldığı şifreyi karakterlerine ayırarak her bir karakteri character dizisine ekliyor. Böylece karakterlerin her birinin birbiri ile olan kombinasyonlarının şifrenin gücünü ne şekilde etkilediğini model görüş oluyor.

In [ ]:
def word(password):
    character=[]
    for i in password:
        character.append(i)
    return character

Bağımsız değişken, bir denklemin veya modelin sonucunu etkileyen değişkendir ve genellikle tahmin etmeye çalıştığımız değeri (bağımlı değişkeni) etkileyen faktördür. Bağımsız değişkenler, tahmin etmeye çalıştığımız sonucu açıklamak veya öngörmek için kullanılan verilerdir.

Makine öğrenimi bağlamında, bağımsız değişkenler modelin girdileridir ve genellikle bu değişkenler kullanılarak bağımlı değişkenin (tahmin edilecek değerin) tahmini yapılır. Yani, bağımsız değişkenlerin belirli bir değeri verildiğinde, bağımlı değişkenin tahmin edilen değeri elde edilir.

Şifreleri baz alarak güç tahminin yapacağımız için password sütununu x değişkenine, strength sütununu ise y değişkenine atıyoruz.

In [ ]:
x = np.array(data["password"])
y = np.array(data["strength"])

Sklearn kütüphanesindeki TfidfVectorizer, metin verilerini temsil etmek için sıklıkla kullanılan bir vektörleştirme aracıdır. Metin verilerini sayısal vektörlere dönüştürmek, makine öğrenimi algoritmalarının doğrudan işleyebileceği bir formata getirir.

TfidfVectorizer özellikle sık kullanılan bir metin madenciliği ve makine öğrenimi aracıdır. İsmi "Term Frequency-Inverse Document Frequency" kısaltmasından gelir. Bu, metin verilerinin her bir teriminin önem derecesini sayısal olarak ifade eden bir istatistiksel ölçüdür.

Term Frequency (TF): Bir belgedeki bir terimin ne kadar sık geçtiğini ifade eder. Daha fazla geçen terimler, genellikle belgenin içeriği hakkında daha fazla bilgi verir.

Inverse Document Frequency (IDF): Bir terimin ne kadar yaygın veya nadir olduğunu ölçer. Eğer bir terim nadirse, yani diğer belgelerde pek geçmiyorsa, bu terimin daha fazla ağırlığı olur. Bu şekilde, nadir terimler daha belirgin olur.

TF-IDF, bir terimin bir belgedeki görülme sıklığını (TF) belirlerken, aynı terimin tüm belgelerdeki genel yaygınlığını (IDF) göz önünde bulundurarak bir terimin belgedeki önemini belirler. Bu, nadir terimlere daha fazla ağırlık verirken, çok yaygın olanlara az ağırlık verir.
TfidfVectorizer kullanıldığında, belgelerin metinlerini vektörlere dönüştürmek için kullanılır ve bu, makine öğrenimi modellerine girdi olarak verilebilir. Bu vektörler, metin verilerinin özelliklerini temsil eder ve çeşitli makine öğrenimi algoritmalarında kullanılabilir. TF-IDF, özellikle belge sınıflandırma, metin kümeleri arasındaki benzerlik hesaplama ve bilgi alaka derecelendirmesi gibi metin odaklı görevlerde yaygın olarak kullanılır.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

TfidfVectorizer kullanırken tokenizer parametresi, metin verilerini vektörlere dönüştürürken kullanılacak olan belirli bir tokenleştirme işlevini belirler. Bu parametre, metinlerin nasıl parçalara ayrılacağını belirtmek için kullanılır. Genellikle, özel bir tokenleştirme işlevi belirtmek için tokenizer parametresini kullanılabilir. Biz burada kendi tokenlaştırma fonksiyonumuzu -word fonksiyonu- tanımladığımız için tokenizer parametresine onu veriyoruz.

In [ ]:
tdif = TfidfVectorizer(tokenizer=word)

Burada tdif.fit_transform(x) kodu, TfidfVectorizer'ı kullanarak belirli bir metin veri kümesini vektörlere dönüştürmek için kullanılır.

fit_transform() yöntemi, önce belirtilen metin veri kümesini TfidfVectorizer'a uyum sağlamak için kullanılır (fit()), ardından bu veri kümesini vektörlere dönüştürmek için kullanılır (transform()).

x genellikle bir liste veya dizide depolanan metin verilerini içerir. Her bir öğe bir belgeyi temsil eder.

Bu kodun çalıştırılması sonucunda, metin verileri TF-IDF vektörlerine dönüştürülür ve x adlı değişkenin içeriği bu dönüştürülmüş vektörlerle güncelleniyor. Bu dönüştürülmüş vektörler, makine öğrenimi modellerine girdi olarak verilebilir. Bu, genellikle sınıflandırma, kümeleme veya diğer metin odaklı görevlerde kullanılır. Bu dönüştürme işlemi, metin verilerini sayısal bir forma dönüştürerek, makine öğrenimi modellerinin bu verileri anlayabilmesini sağlıyor.

In [ ]:
x = tdif.fit_transform(x)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split

 Verilerin eğitim ve test kümelerine bölünmesini sağlar. test_size parametresi, verilerin %5'inin test verisi olarak ayrılacağını belirtir. random_state, rastgele veri bölümleme işleminin yeniden üretilebilirliğini sağlar.

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y,
                                                test_size=0.05,
                                                random_state=42)

Şimdi şifrenin gücünü tahmin etmek için bir sınıflandırma modelini oluşturacagız.

RandomForestClassifier sınıfından bir model oluşturulur. Random Forest sınıflandırıcı, birçok karar ağacının bir araya gelmesiyle oluşturulan bir sınıflandırma algoritmasıdır.

Random Forest algoritması, birçok avantajı nedeniyle bu tür bir problemin çözümünde tercih edilebilir:

Yüksek Performans: Random Forest genellikle yüksek performans sergiler. Büyük veri setlerinde ve yüksek boyutlu veri özelliklerinde bile etkili bir şekilde çalışabilir.

Overfitting'e Karşı Direnç: Diğer karmaşık modellere kıyasla Random Forest, overfitting'e karşı daha dirençlidir. Bu, modelin eğitim verilerine aşırı uyum sağlamayı engeller ve genelleme yeteneğini artırır.

Feature Seçimi ve Önemi Belirleme: Random Forest, her ağaçta bir alt küme özelliklerin rastgele seçilmesi nedeniyle, veri setindeki en önemli özellikleri belirleme konusunda oldukça etkilidir. Bu, modelin daha iyi anlaşılmasını sağlar ve gereksiz özelliklerin çıkarılmasına yardımcı olabilir.

Outlier'lara Karşı Dayanıklılık: Random Forest, outlier'ları işlemek konusunda oldukça iyi bir performans sergiler. Bu, gerçek dünya veri setlerindeki gürültüye veya anormalliklere karşı dirençli olmasını sağlar.

* **Yüksek Doğruluk:** Random Forest, genellikle yüksek doğruluk sağlayan bir sınıflandırma algoritmasıdır.
* **Genelleme Yeteneği:** Random Forest, eğitim verilerine aşırı uyum gösterme eğiliminde değildir ve genelleme yeteneği yüksektir.
* **Özellik Önemliliği:** Random Forest, özelliklerin önemini ölçebilir ve bu bilgi, özellik seçiminde ve modelin anlaşılmasında yardımcı olabilir.
* **Paralel İşleme:** Random Forest, paralel işleme için uygun bir algoritmadır ve büyük veri kümelerinde eğitim ve tahmin işlemlerini hızlandırabilir.
* **Kararlılık:** Random Forest, eğitim verilerindeki küçük değişikliklere karşı kararlıdır ve aşırı uyum gösterme eğiliminde değildir.
* **Veri Tipi Esnekliği:** Random Forest, hem sayısal hem de kategorik özellikleri işleyebilir ve veri ön işleme gereksinimleri nispeten azdır.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RandomForestClassifier'ını kullanabilmek için ondan model nesnesini üretiyoruz.

In [ ]:
model = RandomForestClassifier()

Oluşturulan model, eğitim veri kümesi (xtrain ve ytrain) üzerinde eğitilir. xtrain, özellik vektörlerini, ytrain ise bu özellik vektörlerine karşılık gelen hedef değişken değerlerini içeriyor.

In [ ]:
model.fit(xtrain, ytrain)

RandomForestClassifier()

 Model, test veri kümesi (xtest ve ytest) üzerinde değerlendirilir ve doğruluk puanı (accuracy score) ekrana yazdırılır. Doğruluk puanı, doğru tahmin edilen etiketlerin toplam sayısının, toplam örnek sayısına oranıdır. Bu, modelin test veri kümesindeki performansını ölçmek için kullanılır.

Verilen bir accuracy değerinin iyi veya kötü olduğunu belirlemek, modelin başarısını tam olarak anlamak için yeterli değildir.
Model başarısını daha iyi anlamak için diğer performans ölçütlerini (confusion matrix, precision, recall, F1 score gibi metrikler)de göz önünde bulundurmak lazım. Ayrıca, modelin hiperparametrelerinin nasıl ayarlandığı, veri setinin nasıl bölündüğü gibi faktörler de modelin gerçek performansını etkileyebilir.

In [ ]:
print(model.score(xtest, ytest))

0.9490011167638664


Bu, kullanıcının parolasını gizli bir şekilde girmesini sağlayan getpass modülünü içe aktarıyor.

In [ ]:
import getpass

Kullanıcıya bir parola girmesi istiyoruz. Kullanıcı bu parolayı girerken ekranda görüntülenmez.

In [ ]:
user = getpass.getpass("Enter Password: ")

Enter Password: ··········


Kullanıcının girdiği parola (user), daha önce oluşturulan TfidfVectorizer (tdif) kullanılarak vektör formuna dönüştürüyor. transform() metodu, parolayı vektör haline getirir. toarray() metodu, bu vektörü bir numpy dizisine dönüştürür.

In [ ]:
data = tdif.transform([user]).toarray()

predict() metodu, verilen giriş vektörü üzerinde modelin tahmin yapmasını sağlar.

In [ ]:
output = model.predict(data)

Modelin tahmini sonucu ekrana yazdırıyoruz. Bu, modelin girdi parolasının tahmini gücünü ifade ediyor.

In [ ]:
print(output)

['Weak']
